<i> Александра Березовская, БКЛ-192 </i>

<b> Вариант 3: Пользователь задает стартовую страницу Википедии. Необходимо вытащить все термины, отстоящие по ссылкам от данной страницы не далее некоторого порогового расстояния. </b>

In [134]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import time

Функция, которая вытаскивает ссылки со страницы Википедии (из текста и из инфобоксов)

In [131]:
def SearchUrls(url):
    terms = defaultdict(str)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    article = soup.find(id='bodyContent')
    text = article.find_all('p') #почти ссылки находятся в тегах p, примечания и ссылки под изображениями я не взяла
    for elem in text:
        if elem.find(href=True):
            hrefs = elem.find_all(href=True)
            for piece in hrefs:
                if re.search('title="', str(piece)):
                    href = piece['href']
                    if re.search('\/wiki\/', href):
                        term = piece['title']
                        new_url = 'https://ru.wikipedia.org' + href
                        if not re.search('([\d]{1,2}\s[а-я]{3,8})|([\d]{4})', term):
                            #я очень надеюсь, что под эту регулярку попадут только даты, потому что мне кажется именно так
                            terms[term] = new_url
    if article.find(class_='infobox'):
        box = article.find(class_='infobox')
        hrefs_box = box.find_all(href=True)
        for el in hrefs_box[:-2]:
            if re.search('title="', str(el)):
                href = el['href']
                if re.search('\/wiki\/', href):
                    term = el['title']
                    new_url = 'https://ru.wikipedia.org' + href
                    if not re.search('([\d]{1,2}\s[а-я]{3,8})|([\d]{4})', term):
                        terms[term] = new_url
    elif article.find(class_='toccolours'):
        box = article.find(class_='toccolours')
        hrefs_box = box.find_all(href=True)
        for el in hrefs_box:
            if re.search('title="', str(el)):
                href = el['href']
                if re.search('\/wiki\/', href):
                    term = el['title']
                    new_url = 'https://ru.wikipedia.org' + href
                    if not re.search('([\d]{1,2}\s[а-я]{3,8})|([\d]{1,4})', term):
                        terms[term] = new_url
    time.sleep(0.5)
    return terms

Функция для поиска терминов в зависимости от порогового расстояния n

In [132]:
def SearchEverything(url, n):
    final_terms = defaultdict(str)
    if n == 0:
        final_terms = url
    else:
        i = 1
        terms = SearchUrls(url)
        final_terms.update(terms)
        while i != n:
            new_terms = {}
            for url in terms.values():
                new_terms.update(SearchUrls(url))
            i += 1
            terms = new_terms
            final_terms.update(new_terms)
    return final_terms

Пример, как выводятся термины и ссылки на них:

In [139]:
url = 'https://ru.wikipedia.org/wiki/Телигонум_обыкновенный'
url1 = 'https://ru.wikipedia.org/wiki/Макдональд,_Джон_Александр'
n = 2
SearchEverything(url, n)

defaultdict(str,
            {'L.': 'https://ru.wikipedia.org/wiki/L.',
             'Латинский язык': 'https://ru.wikipedia.org/wiki/%D0%9B%D0%B0%D1%82%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9_%D1%8F%D0%B7%D1%8B%D0%BA',
             'Растения': 'https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%82%D0%B5%D0%BD%D0%B8%D1%8F',
             'Род (биология)': 'https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D0%B4_(%D0%B1%D0%B8%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F)',
             'Телигонум': 'https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BB%D0%B8%D0%B3%D0%BE%D0%BD%D1%83%D0%BC',
             'Мареновые': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%80%D0%B5%D0%BD%D0%BE%D0%B2%D1%8B%D0%B5',
             'Однолетние растения': 'https://ru.wikipedia.org/wiki/%D0%9E%D0%B4%D0%BD%D0%BE%D0%BB%D0%B5%D1%82%D0%BD%D0%B8%D0%B5_%D1%80%D0%B0%D1%81%D1%82%D0%B5%D0%BD%D0%B8%D1%8F',
             'Травянистые растения': 'https://ru.wikipedia.org/wiki/%D0%A2%D1%80%D0%B0%D0%B2%D1%8F%D0%BD%D0%B8%D1%81%D1%82%D1%8B%D0%B5_%